## Render Skywater 130nm Chip GDS File in 3D

### First, run the installation for either Windows or Mac
If Jupyter Notebooks is not installed, run the <a href = "https://canvas.nd.edu/courses/53612/pages/lab-01-hardware-programming">Activity 0 for Lab 1</a> from the CSE 10001 Principles 

### This code assumes you have successfully run the following installations in the Anaconda3 prompt:
<code>pip install numpy</code><br>
<code>pip install gdspy</code><br>
<code>pip install numpy-stl</code><br>
<code>pip install triangle</code><br>
<code>pip install k3d</code><br>

The versions used in this installation were <code>gdspy-1.6.13</code>, <code>numpy-stl-3.0.1</code>, <code>python-utils-3.5.2</code>, <code>triangle-20220202</code>, and <code>k3d-2.15.2</code>.<br>

Next, I ran the <code>jupyter notebook</code> command from the Anaconda prompt

In [1]:
# Require to read files through Jupyter Notebooks
import os

# gdspy is used to open the gds file
import gdspy

# Used to write the output stl file (Why we installed numpy-stl)
from stl import mesh

# Using numpy will permit fast calculations on lots of points
import numpy as np
import matplotlib

# Required to triangulate polygons
import triangle

# To render in 3d
import k3d

In [2]:
# Read the file name for the GDS file 
gdsii_file_path = input('Input gds file name: ')

Input gds file name: Bora-Layout/layout.gds


#### The layerstack sizes are based on the Process Stack Diagram from the Sky130 PDK documentation
<a href = "https://skywater-pdk.readthedocs.io/en/main/rules/assumptions.html">Criteria and Assumptions</a>

![Alt text](https://skywater-pdk.readthedocs.io/en/main/_images/metal_stack.svg "a title")

In [3]:
# choose which GDSII layers to use

# Comment out Skywater Version
layerstack = {
    # (layernumber, datatype) : (zmin, zmax, 'layername'),
    (235,4): (-0.5, 0, 'substrate'),
    (64,20): (-0.24, 0, 'nwell'),    
    (65,44): (-0.12, 0, 'tap'),    
    (65,20): (-0.12, 0, 'diff'),    
    (66,20): (0.02, 0.20, 'poly'),    
    (66,44): (0, 0.9361, 'licon'),    
    (67,20): (0.9361, 1.0111, 'li1'),    
    (67,44): (1.0111, 1.3761, 'mcon'),    
    (68,20): (1.3761, 1.7361, 'met1'),    
    (68,44): (1.7361, 2.0061, 'via'),    
    (69,20): (2.0061, 2.3661, 'met2'),    
    (69,44): (2.3661, 2.7861, 'via2'),    
    (70,20): (2.7861, 3.6311, 'met3'),    
    (70,44): (3.6311, 4.0211, 'via3'),    
    (71,20): (4.0211, 4.8661, 'met4'),    
    (71,44): (4.8661, 5.3711, 'via4'),    
    (72,20): (5.3711, 6.6311, 'met5'),
}

In [4]:
print('Reading GDSII file {}...'.format(gdsii_file_path))
gdsii = gdspy.GdsLibrary()
gdsii.read_gds(gdsii_file_path, units='import')

print('Extracting polygons...')
layers = {} # array to hold all geometry, sorted into layers

cells = gdsii.top_level() # get all cells that aren't referenced by another
for cell in cells: # loop through cells to read paths and polygons

    # $$$CONTEXT_INFO$$$ is a separate, non-standard compliant cell added
    # optionally by KLayout to store extra information not needed here.
    # see https://www.klayout.de/forum/discussion/1026/very-
    # important-gds-exported-from-k-layout-not-working-on-cadence-at-foundry
    if cell.name == '$$$CONTEXT_INFO$$$':
        continue # skip this cell

    # combine will all referenced cells (instances, SREFs, AREFs, etc.)
    cell = cell.flatten()

    # loop through paths in cell
    for path in cell.paths:
        
        # GDSII layer number
        lnum = (path.layers[0],path.datatypes[0])
        
        # create empty array to hold layer polygons if it doesn't yet exist
        layers[lnum] = [] if not lnum in layers else layers[lnum]
        
        # add paths (converted to polygons) that layer
        for poly in path.get_polygons():
            layers[lnum].append((poly, None, False))

    # loop through polygons (and boxes) in cell
    for polygon in cell.polygons:
        lnum = (polygon.layers[0],polygon.datatypes[0]) # same as before...
        layers[lnum] = [] if not lnum in layers else layers[lnum]
        for poly in polygon.polygons:
            layers[lnum].append((poly, None, False))
            
print('Extraction complete.')

Reading GDSII file Bora-Layout/layout.gds...
Extracting polygons...
Extraction complete.


In [5]:
# Get polygon data
print('Number of layers: ' + str(len(layers)))

for cell in cells:
    print( 'Cell Name: ' + str(cell.name) )

# Print all the layer information
for curr_layer in layers:
    print( 'Layer Name: ' + str(curr_layer) )

Number of layers: 36
Cell Name: sramgen_sram
Layer Name: (69, 20)
Layer Name: (70, 20)
Layer Name: (70, 16)
Layer Name: (67, 20)
Layer Name: (68, 20)
Layer Name: (68, 16)
Layer Name: (69, 16)
Layer Name: (81, 2)
Layer Name: (236, 0)
Layer Name: (67, 44)
Layer Name: (66, 20)
Layer Name: (66, 44)
Layer Name: (95, 20)
Layer Name: (92, 44)
Layer Name: (64, 18)
Layer Name: (93, 44)
Layer Name: (65, 44)
Layer Name: (64, 20)
Layer Name: (33, 43)
Layer Name: (68, 44)
Layer Name: (65, 20)
Layer Name: (94, 20)
Layer Name: (22, 21)
Layer Name: (115, 42)
Layer Name: (78, 44)
Layer Name: (115, 43)
Layer Name: (66, 16)
Layer Name: (64, 44)
Layer Name: (122, 16)
Layer Name: (64, 16)
Layer Name: (22, 22)
Layer Name: (69, 44)
Layer Name: (81, 4)
Layer Name: (67, 16)
Layer Name: (125, 44)
Layer Name: (235, 4)


In [6]:
########## TRIANGULATION ######################################################

# An STL file is a list of triangles, so the polygons need to be filled with
# triangles. 
# Documentation: https://rufat.be/triangle/
# Documentation: https://www.cs.cmu.edu/~quake/triangle.html

print('Triangulating polygons...')

num_triangles = {} # will store the number of triangles for each layer

# loop through all layers
for layer_number, polygons in layers.items():

    # but skip layer if it won't be exported
    if not layer_number in layerstack.keys():
        continue

    num_triangles[layer_number] = 0

    # loop through polygons in layer
    for index, (polygon, _, _) in enumerate(polygons):

        num_polygon_points = len(polygon)

        # determine whether polygon points are CW or CCW
        area = 0
        for i, v1 in enumerate(polygon): # loop through vertices
            v2 = polygon[(i+1) % num_polygon_points]
            area += (v2[0]-v1[0])*(v2[1]+v1[1]) # integrate area
        clockwise = area > 0

        # GDSII implements holes in polygons by making the polygon edge
        # wrap into the hole and back out along the same line. However,
        # this confuses the triangulation library, which fills the holes
        # with extra triangles. Avoid this by moving each edge back a
        # very small amount so that no two edges of the same polygon overlap.
        delta = 0.00001 # inset each vertex by this much (smaller has broken one file)
        
        points_i = polygon # get list of points
        points_j = np.roll(points_i, -1, axis=0) # shift by 1
        points_k = np.roll(points_i, 1, axis=0) # shift by -1
        
        # calculate normals for each edge of each vertex (in parallel, for speed)
        normal_ij = np.stack((points_j[:, 1]-points_i[:, 1], points_i[:, 0]-points_j[:, 0]), axis=1)
        normal_ik = np.stack((points_i[:, 1]-points_k[:, 1], points_k[:, 0]-points_i[:, 0]), axis=1)
        
        length_ij = np.linalg.norm(normal_ij, axis=1)
        length_ik = np.linalg.norm(normal_ik, axis=1)
        
        normal_ij /= np.stack((length_ij, length_ij), axis=1)
        normal_ik /= np.stack((length_ik, length_ik), axis=1)
        
        if clockwise:
            normal_ij = -1*normal_ij
            normal_ik = -1*normal_ik
            
        # move each vertex inward along its two edge normals
        polygon = points_i - delta*normal_ij - delta*normal_ik

        hole_delta = 0.00001 # small fraction of delta
        holes = 0.5*(points_j+points_i) - hole_delta*delta*normal_ij
        
        use_holes = False

        # triangulate: compute triangles to fill polygon
        point_array = np.arange(num_polygon_points)
        edges = np.transpose(np.stack((point_array, np.roll(point_array, 1))))
        if use_holes:
            triangles = triangle.triangulate(dict(vertices=polygon,
                                                  segments=edges,
                                                  holes=holes), opts='p')
        else:
            triangles = triangle.triangulate(dict(vertices=polygon,
                                                  segments=edges), opts='p')

        if not 'triangles' in triangles.keys():
            triangles['triangles'] = []

        # each line segment will make two triangles (for a rectangle), and the polygon
        # triangulation will be copied on the top and bottom of the layer.
        num_triangles[layer_number] += num_polygon_points*2 + \
                                       len(triangles['triangles'])*2
        polygons[index] = (polygon, triangles, clockwise)

print('Triangulation complete')

Triangulating polygons...
Triangulation complete


In [7]:
########## EXTRUSION ##########################################################

# Now that we have polygon boundaries and triangulations, we can write it to an STL file.
# Documentation: https://numpy-stl.readthedocs.io/en/latest
print('Extruding polygons and writing to files...')

# Create empty list of STL file names
stl_file_names = []

# loop through all layers
for layer in layers:

    # but skip layer if it won't be exported
    if not layer in layerstack.keys():
        continue

    # Make a list of triangles.
    # This data contains vertex xyz position data as follows:
    # layer_mesh_data['vectors'] = [ [[x1,y1,z1], [x2,y2,z1], [x3,y3,z3]], ...]
    layer_mesh_data = np.zeros(num_triangles[layer], dtype=mesh.Mesh.dtype)

    layer_pointer = 0
    for index, (polygon, triangles, clockwise) in enumerate(layers[layer]):

        # The numpy-stl library expects counterclockwise triangles. That is,
        # one side of each triangle is the outside surface of the STL file
        # object (assuming a watertight volume), and the other side is the
        # inside surface. If looking at a triangle from the outside, the
        # vertices should be in counterclockwise order. Failure to do so may
        # cause certain STL file display programs to not display the
        # triangles correctly (e.g., the backward triangles will be invisible).
        
        zmin, zmax, layername = layerstack[layer]

        # make a list of triangles around the polygon boundary
        points_i = polygon # list of 2D vertices
        
        if clockwise: # order polygon 2D vertices counter-clockwise
            points_i = np.flip(polygon, axis=0)
            
        points_i_min = np.insert(points_i, 2, zmin, axis=1) # bottom left
        points_i_max = np.insert(points_i, 2, zmax, axis=1) # top left
        points_j_min = np.roll(points_i_min, -1, axis=0) # bottom right
        points_j_max = np.roll(points_i_max, -1, axis=0) # top right
        
        rights = np.stack((points_i_min, points_j_min, points_j_max), axis=1)
        lefts = np.stack((points_j_max, points_i_max, points_i_min), axis=1)

        # make a list of polygon interior (face) triangles
        vs = triangles['vertices']
        ts = triangles['triangles']
        
        if len(ts) > 0:
            face_tris = np.take(vs, ts, axis=0)
            top = np.insert(face_tris, 2, zmax, axis=2) # list of top triangles
            bottom = np.insert(face_tris, 2, zmin, axis=2) # list of bottom ~
            bottom = np.flip(bottom, axis=1) # reverse vertex order to make CCW
            faces = np.concatenate((lefts, rights, top, bottom), axis=0)
            
        else: # didn't generate any triangles! (degenerate edge case)
            faces = np.concatenate((lefts, rights), axis=0)

        # add side and face triangles to layer mesh
        layer_mesh_data['vectors'][layer_pointer:(layer_pointer+len(faces))] = faces
        layer_pointer += len(faces)

    # save layer to STL file
    filename = gdsii_file_path + '_{}.stl'.format(layername)
    print('    ({}, {}) to {}'.format(layer, layername, filename))
    layer_mesh_object = mesh.Mesh(layer_mesh_data, remove_empty_areas=False)
    layer_mesh_object.save(filename)
    
    # Add STL file to list
    stl_file_names.append(filename)

print('Done.')

Extruding polygons and writing to files...
    ((69, 20), met2) to Bora-Layout/layout.gds_met2.stl
    ((70, 20), met3) to Bora-Layout/layout.gds_met3.stl
    ((67, 20), li1) to Bora-Layout/layout.gds_li1.stl
    ((68, 20), met1) to Bora-Layout/layout.gds_met1.stl
    ((67, 44), mcon) to Bora-Layout/layout.gds_mcon.stl
    ((66, 20), poly) to Bora-Layout/layout.gds_poly.stl
    ((66, 44), licon) to Bora-Layout/layout.gds_licon.stl
    ((65, 44), tap) to Bora-Layout/layout.gds_tap.stl
    ((64, 20), nwell) to Bora-Layout/layout.gds_nwell.stl
    ((68, 44), via) to Bora-Layout/layout.gds_via.stl
    ((65, 20), diff) to Bora-Layout/layout.gds_diff.stl
    ((69, 44), via2) to Bora-Layout/layout.gds_via2.stl
    ((235, 4), substrate) to Bora-Layout/layout.gds_substrate.stl
Done.


In [8]:
# Render layered STL files from Extraction, Triangulation, and Extrusion

full_plot = k3d.plot()

color = [0xe3dac9, 0xff0000, 0x00ff00, 0x0000ff, 0xff00ff, 0x00ffff, 0xffff00, 0x330000, 0x003300, 0x000033, 0x330033, 0x333300, 0x003333, 0x990000, 0x009900, 0x000099, 0x990099, 0x999900, 0x009999]

for i in range(len(stl_file_names)):

    with open(stl_file_names[i], 'rb') as stl:
        data = stl.read()

    plt_skull = k3d.stl(data, color[i])

    full_plot += plt_skull

full_plot.display()

Output()